## Libraries

In [1]:
import tempfile
import math
import os
from subprocess import call
import matplotlib.pyplot as plt
from f90nml import *
from array import *
import numpy as np
import pandas as pd 
import seaborn as sns
from statistics import *
import shutil
import sys
import time
from matplotlib.ticker import FormatStrFormatter

#read in printFolder, makeDefNameList, newFolder, deleteFolder, readOutput from coreFunctions.py
from coreFunctions import *

#read in plotting functionality from plottingFunctions.py
from plottingFunctions import *

call("rm -rf tmp*",shell=True)

dfModel = []#will contain output from our model

## Enter Custom Inputs, then Run the Model

In [4]:
nameList = makeDefNamelist() #make namelist
nameList['ebm']['lverbose']=False #controls terminal output, used for debugging
nameList['ebm']['rBirth']=.0252;
nameList['ebm']['rDeath']=.0142857;
nameList['ebm']['rco2']=2.00e-4;
life=False;

start = time.time()

#specify inputs
multP=1
runTime=2000#change runtime(years)
coupled=True
saveName=1;#name of file
fullMaxPop=0
save=True;#save the file
for j in np.linspace(284*.1,284*1000,5):#pco2 loops
    newPco2=int(j)#change pco2 (ppm)

    newA = .9#change distance (AU)
    dA=.01
    minA, maxA, fullMaxPop, fullMaxPopA = habitableZone(nameList,newPco2,newA,runTime,dA)
    print("The Habitable Zone if pCO2="+str(round(newPco2)) + "ppm is:")
    print("Minimum: "+str(minA) + " AU")
    print("Maximum: "+str(maxA)+" AU")
    print("Max Pop: " + str(fullMaxPop)+ " billion" + ", at Distance: " + str(round(fullMaxPopA,3))+"\n") 

    for i in np.linspace(minA,maxA,4):#population loops
    #calculate inputs
        newA = round(i,2) #change distance (AU)
        nameList['ebm']['coupled']=coupled
        nameList['ebm']['pco20']=newPco2/10**6#convert pco2 to bars
        nameList['ebm']['relsolcon']=newA**-2 #inverse square law for solar flux
        nameList['ebm']['runTime'] = runTime#change runtime
    #run program
        dfModel, finalavgtemp, eqTime, eqTemp, equilibrium = runProgram("driver.exe",nameList,True)#False=no output
    #plot the results
        if coupled:
            popStats = analyzeRun(dfModel,nameList,False)#True Prints Dictionary Values
            popStats['maxPopPlot']=fullMaxPop+(3/100)*fullMaxPop#maximum population range
            inputs=[newA,newPco2,runTime]
       #     if equilibrium: plotModelOutput(dfModel,inputs,eqTime,eqTemp,popStats,save,saveName)#plot the output of our model, colored by pco2 
            saveName+=1
    end = time.time()
    print( "Elapsed Time: " + str(end-start))
    call("echo End of Python Notebook Reached", shell=True);

The Habitable Zone if pCO2=28ppm is:
Minimum: 0.93 AU
Maximum: 1.0 AU
Max Pop: 3.2531 billion, at Distance: 0.98

Equilibrium Reached at Temp=337.141. At time=96.0
Final Temp(K): 439.16
Final Temp(F): 330.82

Equilibrium Reached at Temp=310.701. At time=42.0
Final Temp(K): 319.47
Final Temp(F): 115.38

Equilibrium Reached at Temp=289.877. At time=25.0
Final Temp(K): 297.68
Final Temp(F): 76.15

Equilibrium Reached at Temp=278.369. At time=24.0
Final Temp(K): 286.09
Final Temp(F): 55.29

Elapsed Time: 58.632619857788086
The Habitable Zone if pCO2=71021ppm is:
Minimum: 0.93 AU
Maximum: 1.12 AU
Max Pop: 23294.0 billion, at Distance: 0.93

Equilibrium Reached at Temp=368.984. At time=42.0
Final Temp(K): 378.51
Final Temp(F): 221.65

Equilibrium Reached at Temp=330.667. At time=35.0
Final Temp(K): 339.23
Final Temp(F): 150.94

Equilibrium Reached at Temp=299.134. At time=27.0
Final Temp(K): 307.98
Final Temp(F): 94.69

Equilibrium Reached at Temp=277.092. At time=27.0
Final Temp(K): 285.97


## Model Analysis

In [3]:
#dfTemp = pd.read_csv('anomaly.csv');#data from NASA GISS: https://data.giss.nasa.gov/gistemp/graphs_v4/
#dfPopCo2 = pd.read_csv('world_stats.csv');#data from Frank, Adam, and Woodruff Sullivan.
#plotModelInput(nameList)#plot the temperature dependence on the growth rate
#plotTruePopCo2(dfPopCo2) #plot true population values vs pco2
#compareModelOutput(dfModel,dfTemp,dfPopCo2)#compare the models output to true data